In [22]:
import scipy.io as sio
import numpy as np
import os
import fileinput
import plotly.graph_objs as go
import sklearn
import struct
import re
import csv
import pickle
import pandas as pd
from scipy.cluster.vq import whiten
from sklearn import cluster
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn import decomposition
import matplotlib.pyplot as plt
import urllib
from matplotlib.backends.backend_pdf import PdfPages
import urllib3
from sklearn.cross_decomposition import CCA
import plotly.plotly as py
import http
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles
import plotly.graph_objs as go
from numpy import genfromtxt
%matplotlib inline

# Load and format the data

In [13]:
# Load the data from the Google cloud for FOLD_3. (ON THE GOOGLE CLOUD STORAGE)

# Import the name of files from virtual machine

my_data = genfromtxt('FOLD3.txt', delimiter='\n', dtype=str)

url =  "https://storage.googleapis.com/bucket-pinouche/ImperialData/FOLD_3"

dic = {}
number_of_files = 0

for filename in my_data:
    print(filename)
    f = urllib.request.urlopen(url+'/'+filename)
    with open(os.path.basename(url), "wb") as local_file:
        local_file.write(f.read())
        dic[filename] = sio.loadmat(local_file.name)
    
    number_of_files = number_of_files + 1

'file:///C:/Users/pinouche/Documents/ImperialData/FOLD_3'

In [ ]:
# The Annotation folder is stored on the virtual machine instance directly (ON THE VIRTUAL MACHINE)

even_names_files = list()
index = 0

for name in name_of_files:
    if(index%2 == 0):
        even_names_files.append(name[:-10])
    index += 1

dic2 = {}
number_of_files = 0
path = r'/home/uriot_thomas/two'

for filename in even_names_files:
    dic2[filename] = sio.loadmat(path+'/'+filename+'/'+'meanAnnotation.mat')
    number_of_files = number_of_files + 1
    
sequences_shape = list()
mean_annotations = list()

for filename in even_names_files:
    mean_annotations.append(dic2[filename]['annotations'])
    sequences_shape.append(dic2[filename]['annotations'].shape[0])

array_annotations_FOLD_3 = np.concatenate(np.asarray(mean_annotations), axis=0)

print([number_of_files, array_annotations_FOLD_3.shape])

In [14]:
# Getting the features data into usable matrix format

def dataMatrix(array1, array2):

    person1 = list()
    person2 = list()

    for frame in range(0, array1.shape[2]):
        for sift in range(0,49):
            person1.append(array1[sift,:,frame])
            person2.append(array2[sift,:,frame])

    tmp_arr_mat1 = np.reshape(np.asarray(person1),(array1.shape[2],6272))
    tmp_arr_mat2 = np.reshape(np.asarray(person2),(array2.shape[2],6272))
    frame_vector = np.concatenate([tmp_arr_mat1,tmp_arr_mat2], axis=1)
    
    return(frame_vector)


[40, ['20120213_seq1_01_01.mat', '20120213_seq1_01_02.mat', '20120213_seq2_01_01.mat', '20120213_seq2_01_02.mat', '20120326_seq3_01_01.mat', '20120326_seq3_01_02.mat', '20120423_seq10_01_01.mat', '20120423_seq10_01_02.mat', '20120423_seq11_01_01.mat', '20120423_seq11_01_02.mat', '20120423_seq1_01_01.mat', '20120423_seq1_01_02.mat', '20120423_seq2_01_01.mat', '20120423_seq2_01_02.mat', '20120423_seq3_01_01.mat', '20120423_seq3_01_02.mat', '20120423_seq4_01_01.mat', '20120423_seq4_01_02.mat', '20120423_seq6_01_01.mat', '20120423_seq6_01_02.mat', '20120423_seq8_01_01.mat', '20120423_seq8_01_02.mat', '20120423_seq9_01_01.mat', '20120423_seq9_01_02.mat', '20120430_seq10_01_01.mat', '20120430_seq10_01_02.mat', '20120430_seq11_01_01.mat', '20120430_seq11_01_02.mat', '20120430_seq12_01_01.mat', '20120430_seq12_01_02.mat', '20120430_seq2_01_01.mat', '20120430_seq2_01_02.mat', '20120430_seq3_01_01.mat', '20120430_seq3_01_02.mat', '20120430_seq5_01_01.mat', '20120430_seq5_01_02.mat', '20120430_se

In [15]:
# Get all the values from the SIFT values for each of the frame from the dictionnary

key=0
concat = []

for val in name_of_files:
    if(key%2 == 0):
        person1 = dic[val]['SIFT']
    else:
        person2 = dic[val]['SIFT']
        concat.append(dataMatrix(person1, person2))
    key +=1
        
Big_list = np.concatenate(concat)
print(Big_list.shape)

# The dimensions of the Big_list (data matrix) is 20640 by 12544

20


In [16]:
# Instead of concatenating together the vectors for the 2 people, use the average for the data matrix. Note that later on,
# the response we have for each frame is the same for the 2 people (i.e there is a conflict intensity for the overall 
# interaction and not a seperated conflict value for each of the interlocutors). So this approach may be a better way.

Person1_columns = Big_list[:,:(49*128)]
Person2_columns = Big_list[:,(49*128):]
print([Person1_columns.shape,Person2_columns.shape])
Average_Big_list_FOLD_3 = (Person1_columns + Person2_columns)/2

# Peform PCA on the data

In [ ]:
# Plots the annotation for a whole video sequence as an example

mean_contents1 = array_annotations_FOLD_3[:len(dic['20120213_seq1_01_01.mat']['SIFT'][1][1])]


range_array = np.asarray(list(range(0, len(dic['20120213_seq1_01_01.mat']['SIFT'][1][1]), 1)))

fig = plt.figure(figsize=(17, 6))
plt.plot(range_array , mean_contents1, 'ro')
plt.axis([0,len(dic['20120213_seq1_01_01.mat']['SIFT'][1][1]), 0, 1])
fig.suptitle('Ground truth conflict intensity between the 2 participants')
plt.xlabel('Number of frames')
plt.ylabel('Conflict intensity')
plt.show()
fig.savefig('meanplot.png', bbox_inches='tight')

In [ ]:
# whiten the data 

whitened_data = whiten(Average_Big_list)

def svd_whiten(X):
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    X_white = np.dot(U, Vt)

    return X_white

whiten_Big_list = svd_whiten(Average_Big_list_FOLD_3)

In [ ]:
# PCA on the new matrix

n_components = 500
svd = decomposition.TruncatedSVD(n_components=n_components, algorithm='arpack')
svd.fit(Average_Big_list_FOLD_3)
print(svd.explained_variance_ratio_.sum())

# 500 components: 81%

In [ ]:
# Get the new data of dimension  (number of frames * 500)

svd_data = np.dot(Average_Big_list_FOLD_3,np.transpose(svd.components_))

In [ ]:
# Using t-sne on the PCA data to visualize the data (only on 10000 data points for speed purposes)

random_indices = np.random.choice(svd_data.shape[0], 10000, replace=False)
sampled_rows = svd_data[random_indices, :]
sampled_annotations = array_annotations_FOLD_3[random_indices, :]
tsne_svd = manifold.TSNE(n_components=2, verbose=0, perplexity=30, n_iter=2500) 
tsne_results_svd = tsne_svd.fit_transform(sampled_rows) 


In [ ]:
kmeans = cluster.KMeans(n_clusters=3) # k-means
kmeans.fit(sampled_rows)

c = kmeans.labels_
x = tsne_results_svd[:,0]
y = tsne_results_svd[:,1]
#z = tsne_results_svd[:,2]
t = [str(s[0]) for s in sampled_annotations]

trace1 = go.Scatter(x=x,y=y,text=t, mode='markers',marker=dict(size=12,color=c, colorscale = 'Viridis', opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE PCA with k-Means')

# Kernel PCA

In [ ]:
# Transform the data with an RBF Gaussian kernal and perform PCA. Could again use k-means and plot the data in 2 or 3 dimensions
# using t-SNE as was done for PCA above.


n_components = 500
kpca = KernelPCA(kernel="rbf",n_components=n_components, gamma=10)
X_kpca = kpca.fit_transform(Average_Big_list_FOLD_3)

In [ ]:
# Get the eigenvalues

kpca.lambdas_

# Canonical correlation analysis

In [ ]:
# Loading the name of files of FOLD_4

my_data_Fold_4 = genfromtxt('FOLD4.txt', delimiter='\n', dtype=str)

In [ ]:
# Load the data from the Google cloud for FOLD_4. 
# Note that the variable name_of_files is the same one we define in a couple of cells below. (ON THE GOOGLE CLOUD STORAGE)

url =  "https://storage.googleapis.com/bucket-pinouche/ImperialData/FOLD_4"

dic = {}
number_of_files = 0

for filename in my_data_Fold_4 :
    print(filename)
    f = urllib.request.urlopen(url+'/'+filename)
    with open(os.path.basename(url), "wb") as local_file:
        local_file.write(f.read())
        dic[filename] = sio.loadmat(local_file.name)
    
    number_of_files = number_of_files + 1

In [ ]:
# Create a similar matrix to Average_big_list but using the data FOLD_4. This is a sueprvised methods (it uses the labels) and
# we do not want to use the labels to transform our data which will then be trained using the same labels (introduces bias).

# Get all the values from the SIFT values for each of the frame from the dictionnary

key=0
concat = []

for val in my_data_Fold_4:
    if(key%2 == 0):
        person1 = dic[val]['SIFT']
    else:
        person2 = dic[val]['SIFT']
        concat.append(dataMatrix(person1, person2))
    key +=1
        
Big_list = np.concatenate(concat)
print(Big_list.shape)


In [ ]:
# Take the average for the two people

Person1_columns = Big_list[:,:(49*128)]
Person2_columns = Big_list[:,(49*128):]
print([Person1_columns.shape,Person2_columns.shape])
Average_Big_list_FOLD_4 = (Person1_columns + Person2_columns)/2

In [ ]:
# The Annotation folder is stored on the virtual machine instance directly (ON THE VIRTUAL MACHINE) for FOLD_4

even_names_files = list()
index = 0

for name in my_data_Fold_4:
    if(index%2 == 0):
        even_names_files.append(name[:-10])
    index += 1

dic2 = {}
number_of_files = 0
path = r'/home/uriot_thomas/two'

for filename in even_names_files:
    dic2[filename] = sio.loadmat(path+'/'+filename+'/'+'meanAnnotation.mat')
    number_of_files = number_of_files + 1
    
sequences_shape = list()
mean_annotations = list()

for filename in even_names_files:
    mean_annotations.append(dic2[filename]['annotations'])
    sequences_shape.append(dic2[filename]['annotations'].shape[0])

array_annotations_FOLD_4 = np.concatenate(np.asarray(mean_annotations), axis=0)

print([number_of_files, array_annotations_FOLD_4.shape])

In [ ]:
# Perform CCA

cca = CCA(n_components=500)
cca.fit(Average_Big_list_FOLD_4, array_annotations_FOLD_4)  # THIS IS THE TRAINING PART WHICH NEEDS TO BE DONE ON A DIFFERENT FOLD
print(cca.coef_.shape)   

In [ ]:
# Transform the data

cca_data = cca.transform(Average_Big_list) # HERE THIS IS THE USUAL DATA ON FOLD_3
print(cca_data) 